In [1]:
# 使用LSTM模型预测股票价格
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import nn
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# use pytorch to build LSTM model
# 定义LSTM模型
class LSTM(nn.Module):
    def __init__(self, input_size=24, hidden_size=64, num_layers=2, output_size=1, dropout=0.5):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.lstm1 = nn.LSTM(input_size, hidden_size*2, num_layers, dropout=dropout, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size*2, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.linear1 = nn.Linear(hidden_size, 32)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(32, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input):
        output, _ = self.lstm1(input)
        output, _ = self.lstm2(output)
        output, _ = self.lstm3(output)
        output = self.linear1(output[:, -1, :])
        output = self.relu(output)
        output = self.linear2(output)
        output = self.softmax(output)
        return output.squeeze(-1)

# 实例化模型并定义优化器和损失函数
model = LSTM().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [2]:
# 定义函数将数据集转换为带有时间步长的数据集
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :-2]
        dataX.append(a)
        dataY.append(dataset[i + look_back, -2].astype('int'))
    return np.array(dataX), np.array(dataY)

In [4]:

num_epochs = 1000
train_losses = []
from torch.utils.data import DataLoader, TensorDataset
from addons.stock_manage.service import StockDataLoader, Stocks
stocks = Stocks().get_stocks()
for epoch in range(num_epochs):
    for stock in stocks:
        d = StockDataLoader(stock, '2000-01-01', '2023-03-31')
        res = d.get_all_stock_data()
        d.add_ema(res)
        d.add_target(res)
        data = res.iloc[:, 2:]
        data = data.astype('float64')
        data = data.to_numpy()
        # 分割数据集
        train_size = int(len(data) * 0.7)
        test_size = len(data) - train_size
        train_data, test_data = data[0:train_size,:], data[train_size:,:]

        look_back = 10
        trainX, trainY = create_dataset(train_data, look_back)
        testX, testY = create_dataset(test_data, look_back)

        # 将数据集从numpy数组转换为张量
        trainX = torch.from_numpy(trainX).type(torch.Tensor).to(device)
        trainY = torch.from_numpy(trainY).type(torch.Tensor).to(device)
        testX = torch.from_numpy(testX).type(torch.Tensor).to(device)
        testY = torch.from_numpy(testY).type(torch.Tensor).to(device)

        train_dataset = TensorDataset(trainX, trainY)
        # 定义DataLoader，设置batch_size和shuffle参数
        train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
        print("stock: ", stock, " epoch: ", epoch, " loss: ", loss.item())
    if epoch % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))





stock:  000001.SZ  epoch:  0  loss:  3.142857313156128
stock:  000002.SZ  epoch:  0  loss:  2.1052632331848145
stock:  000004.SZ  epoch:  0  loss:  1.8823529481887817
stock:  000005.SZ  epoch:  0  loss:  2.25
stock:  000006.SZ  epoch:  0  loss:  1.5
stock:  000007.SZ  epoch:  0  loss:  2.2857143878936768
stock:  000008.SZ  epoch:  0  loss:  2.322580575942993
stock:  000009.SZ  epoch:  0  loss:  1.8399999141693115
stock:  000010.SZ  epoch:  0  loss:  2.090909242630005
stock:  000011.SZ  epoch:  0  loss:  2.090909242630005
stock:  000012.SZ  epoch:  0  loss:  2.25
stock:  000014.SZ  epoch:  0  loss:  2.305084705352783
stock:  000016.SZ  epoch:  0  loss:  1.8181817531585693
stock:  000017.SZ  epoch:  0  loss:  1.6727272272109985
stock:  000019.SZ  epoch:  0  loss:  1.5384615659713745
stock:  000020.SZ  epoch:  0  loss:  1.91304349899292
stock:  000021.SZ  epoch:  0  loss:  2.0487804412841797
stock:  000023.SZ  epoch:  0  loss:  1.8399999141693115
stock:  000025.SZ  epoch:  0  loss:  1.230

KeyboardInterrupt: 

Epoch [1/1000], Loss: 1.7143
Epoch [101/1000], Loss: 1.1429
Epoch [201/1000], Loss: 1.4286
Epoch [301/1000], Loss: 1.1429
Epoch [401/1000], Loss: 3.1429
Epoch [501/1000], Loss: 3.1429
Epoch [601/1000], Loss: 1.7143
Epoch [701/1000], Loss: 1.4286
Epoch [801/1000], Loss: 1.1429
Epoch [901/1000], Loss: 1.7143


tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
      